<a href="https://colab.research.google.com/github/mar-i0/AI-Notebooks/blob/main/Custom_LLM_Model_for_INDEXING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Instalamos las dependencias necesarias:



In [1]:
!pip install torch
!pip install langchain
!pip install llama_index
!pip install transformers
!pip install typing
%pip install accelerate>=0.12.0 transformers[torch]==4.25.1



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


2. Importamos las librerias necesarias

In [2]:
import torch
from langchain.llms.base import LLM
from llama_index import SimpleDirectoryReader, LangchainEmbedding, GPTListIndex, PromptHelper
from llama_index import LLMPredictor, ServiceContext
from transformers import pipeline
from typing import Optional, List, Mapping, Any


3. Definimos las variables necesarias

In [3]:
# define prompt helper
# set maximum input size
max_input_size = 2048
# set number of output tokens
num_output = 256
# set maximum chunk overlap
max_chunk_overlap = 20
prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap)


4. Definimos las funciones personalizadas necesarias

In [4]:
!nvidia-smi

Mon Apr 17 15:17:06 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    42W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
class CustomLLM(LLM):
    #model_name = "facebook/opt-iml-max-30b"
    #pipeline = pipeline("text-generation", model=model_name, device="cuda:0", model_kwargs={"torch_dtype":torch.bfloat16})

    model_name = "databricks/dolly-v2-12b"
    pipeline = pipeline(model=model_name, torch_dtype=torch.bfloat16, trust_remote_code=True, device_map="auto", device=0)
    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        prompt_length = len(prompt)
        response = self.pipeline(prompt, max_new_tokens=num_output)[0]["generated_text"]

        # only return newly generated tokens
        return response[prompt_length:]

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {"name_of_model": self.model_name}

    @property
    def _llm_type(self) -> str:
        return "custom"

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ in pydantic.main.ModelMetaclass.__new__:221                                                      │
│                                                                                                  │
│ in pydantic.fields.ModelField.infer:506                                                          │
│                                                                                                  │
│ in pydantic.fields.ModelField.__init__:436                                                       │
│                                                                                                  │
│ in pydantic.fields.ModelField.prepare:546                                                        │
│                                                                                                  │
│ in pydantic.fields.ModelField._set_default_and_type:570                                          │
│                                                                                                  │
│ in pydantic.fields.ModelField.get_default:439                                                    │
│                                                                                                  │
│ in pydantic.utils.smart_deepcopy:693                                                             │
│                                                                                                  │
│ /usr/lib/python3.9/copy.py:172 in deepcopy                                                       │
│                                                                                                  │
│   169 │   │   │   │   if isinstance(rv, str):                                                    │
│   170 │   │   │   │   │   y = x                                                                  │
│   171 │   │   │   │   else:                                                                      │
│ ❱ 172 │   │   │   │   │   y = _reconstruct(x, memo, *rv)                                         │
│   173 │                                                                                          │
│   174 │   # If is its own copy, don't memoize.                                                   │
│   175 │   if y is not x:                                                                         │
│                                                                                                  │
│ /usr/lib/python3.9/copy.py:270 in _reconstruct                                                   │
│                                                                                                  │
│   267 │                                                                                          │
│   268 │   if state is not None:                                                                  │
│   269 │   │   if deep:                                                                           │
│ ❱ 270 │   │   │   state = deepcopy(state, memo)                                                  │
│   271 │   │   if hasattr(y, '__setstate__'):                                                     │
│   272 │   │   │   y.__setstate__(state)                                                          │
│   273 │   │   else:                                                                              │
│                                                                                                  │
│ /usr/lib/python3.9/copy.py:146 in deepcopy                                                       │
│                                                                                                  │
│   143 │                                                    

5. Ejecutamos

In [ ]:
# define our LLM
llm_predictor = LLMPredictor(llm=CustomLLM())

service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, prompt_helper=prompt_helper)

# Load the your data
documents = SimpleDirectoryReader('./data').load_data()
index = GPTListIndex.from_documents(documents, service_context=service_context)

# Query and print response
response = new_index.query("<query_text>")
print(response)